In [1]:
import os
import cv2
import numpy as np

In [4]:
import tensorflow as tf
import numpy as np

def load(imagePaths):
    data = []

    for (i,imagePath) in enumerate(imagePaths):
        #label = imagePath.split(os.path.sep)[-2]
        imageFileName = imagePath.split(os.path.sep)[-1]
        label = imageFileName.split('.')[0]
        data.append([imageFileName,label])

    return data

In [2]:
from imutils import paths

imagePaths = list(paths.list_images("train/train"))
imagePaths

['train/train/cat.11863.jpg',
 'train/train/dog.10585.jpg',
 'train/train/cat.2132.jpg',
 'train/train/cat.3700.jpg',
 'train/train/cat.4322.jpg',
 'train/train/dog.10380.jpg',
 'train/train/cat.9901.jpg',
 'train/train/cat.8761.jpg',
 'train/train/dog.4064.jpg',
 'train/train/cat.3744.jpg',
 'train/train/dog.12489.jpg',
 'train/train/cat.208.jpg',
 'train/train/dog.6502.jpg',
 'train/train/cat.11284.jpg',
 'train/train/cat.8357.jpg',
 'train/train/cat.6918.jpg',
 'train/train/cat.7224.jpg',
 'train/train/cat.8363.jpg',
 'train/train/cat.3116.jpg',
 'train/train/cat.5912.jpg',
 'train/train/dog.6736.jpg',
 'train/train/dog.10111.jpg',
 'train/train/cat.9996.jpg',
 'train/train/cat.9609.jpg',
 'train/train/dog.8428.jpg',
 'train/train/dog.2367.jpg',
 'train/train/cat.4156.jpg',
 'train/train/cat.12322.jpg',
 'train/train/dog.2345.jpg',
 'train/train/dog.3459.jpg',
 'train/train/cat.9285.jpg',
 'train/train/dog.911.jpg',
 'train/train/dog.9135.jpg',
 'train/train/dog.11945.jpg',
 'train/

In [5]:
import pandas as pd

df = pd.DataFrame(load(imagePaths),columns=['Filename','Label'])
df.head()

,Filename,Label
0,cat.11863.jpg,cat
1,dog.10585.jpg,dog
2,cat.2132.jpg,cat
3,cat.3700.jpg,cat
4,cat.4322.jpg,cat


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df)

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=30,rescale=1./255,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [11]:
train_generator = datagen.flow_from_dataframe(train,'train/train',x_col='Filename',y_col='Label',target_size=(224,224),batch_size=32)

Found 18750 validated image filenames belonging to 2 classes.


In [12]:
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(test,'train/train',x_col='Filename',y_col='Label',target_size=(224,224),batch_size=32)

Found 6250 validated image filenames belonging to 2 classes.


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
inputShape = (224,224,3)
#if K.image_data_format()=="channels_first":
#    inputShape = (depth,height,width)
model.add(layers.Conv2D(32,(3,3),padding="same",input_shape=inputShape))
model.add(layers.Activation("relu"))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation="softmax"))
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.005),loss="binary_crossentropy",metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 activation_1 (Activation)   (None, 224, 224, 32)      0         
                                                                 
 flatten_1 (Flatten)         (None, 1605632)           0         
                                                                 
 dense_1 (Dense)             (None, 2)                 3211266   
                                                                 
Total params: 3,212,162
Trainable params: 3,212,162
Non-trainable params: 0
_________________________________________________________________


In [15]:
H = model.fit_generator(train_generator,epochs=5,validation_data=val_generator,validation_steps=test.shape[0]//32,steps_per_epoch=train.shape[0]//32)

<ipython-input-15-f9b88a40bde8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_generator,epochs=5,validation_data=val_generator,validation_steps=test.shape[0]//32,steps_per_epoch=train.shape[0]//32)


Epoch 1/5
585/585 [==============================] - 850s 1s/step - loss: 0.7031 - accuracy: 0.5972 - val_loss: 0.6781 - val_accuracy: 0.5835
Epoch 2/5
585/585 [==============================] - 551s 941ms/step - loss: 0.6551 - accuracy: 0.6207 - val_loss: 0.6449 - val_accuracy: 0.6482
Epoch 3/5
585/585 [==============================] - 552s 943ms/step - loss: 0.6428 - accuracy: 0.6395 - val_loss: 0.6309 - val_accuracy: 0.6572
Epoch 4/5
585/585 [==============================] - 543s 927ms/step - loss: 0.6348 - accuracy: 0.6489 - val_loss: 0.6212 - val_accuracy: 0.6625
Epoch 5/5
585/585 [==============================] - 502s 857ms/step - loss: 0.6301 - accuracy: 0.6516 - val_loss: 0.6131 - val_accuracy: 0.6744


In [29]:
y_pred = model.predict_generator(val_generator)

<ipython-input-29-72b5de6470cb>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(val_generator)


In [30]:
y_pred

array([[0.3330152 , 0.6669848 ],
       [0.8471592 , 0.15284072],
       [0.35073972, 0.64926034],
       ...,
       [0.794392  , 0.20560789],
       [0.56170696, 0.438293  ],
       [0.51628095, 0.48371908]], dtype=float32)

In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
labels_transformed = ohe.fit_transform(test['Label'].values.reshape(-1,1))
labels_transformed

<6250x2 sparse matrix of type '<class 'numpy.float64'>'
	with 6250 stored elements in Compressed Sparse Row format>

In [20]:
labels_transformed.toarray()

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [25]:
from sklearn.metrics import classification_report

print(classification_report(labels_transformed.toarray().argmax(axis=1),y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.50      0.51      0.50      3120
           1       0.50      0.49      0.50      3130

    accuracy                           0.50      6250
   macro avg       0.50      0.50      0.50      6250
weighted avg       0.50      0.50      0.50      6250



In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels_transformed = le.fit_transform(test['Label'].values.reshape(-1,1))
labels_transformed

/home/arindam/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 0, ..., 0, 0, 1])

In [33]:
y_final = []
for x in y_pred:
    if x[0]>x[1]:
        y_final.append(1)
    else:
        y_final.append(0)
np.array(y_final)

array([0, 1, 0, ..., 1, 1, 1])

In [34]:
print(classification_report(y_final,labels_transformed))

              precision    recall  f1-score   support

           0       0.49      0.50      0.49      3077
           1       0.51      0.50      0.50      3173

    accuracy                           0.50      6250
   macro avg       0.50      0.50      0.50      6250
weighted avg       0.50      0.50      0.50      6250

